In [2]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import json
from pprint import pprint
import requests
import time
from scipy import stats

# Incorporated citipy to determine city based on latitude and longitude
from citipy import citipy

# Study data files
stanford_path = "Stanford Data.csv"

stanford_df = pd.read_csv(stanford_path)

In [3]:
# Preview Data Frame
stanford_df.head()

,study_id,redcap_survey_identifier,introduction_timestamp,country,state_in_india,state_in_us,province_in_china,state_other_countries,age,race_and_ethnicity,...,sector_of_contribution___4,sector_of_contribution___5,other,where_helped___1,where_helped___2,where_helped___3,where_helped___4,where_helped___5,other_where_helped,social_contributions_complete
0,1,NaN,[not completed],244.0,NaN,NaN,NaN,NaN,NaN,NaN,...,0,0,NaN,0,0,0,0,0,NaN,0
1,2,NaN,4/16/20 14:47,244.0,NaN,NaN,NaN,NaN,2.0,NaN,...,0,0,NaN,0,0,0,0,0,NaN,0
2,3,NaN,4/16/20 14:51,244.0,NaN,NaN,NaN,NaN,3.0,NaN,...,0,0,NaN,0,0,0,0,0,NaN,0
3,4,NaN,4/16/20 14:53,9.0,NaN,NaN,NaN,NaN,3.0,NaN,...,0,0,NaN,0,0,0,0,0,NaN,0
4,5,NaN,4/16/20 15:01,2.0,NaN,NaN,NaN,NaN,5.0,NaN,...,0,0,NaN,0,0,0,1,0,NaN,2


In [12]:
# Delete incomplete surveys
#drop any rows that have 7 in the rebounds column
stanford_df = stanford_df[stanford_df.introduction_timestamp != "[not completed]"]

#view resulting DataFrame
stanford_df

,study_id,redcap_survey_identifier,introduction_timestamp,country,state_in_india,state_in_us,province_in_china,state_other_countries,age,race_and_ethnicity,...,sector_of_contribution___4,sector_of_contribution___5,other,where_helped___1,where_helped___2,where_helped___3,where_helped___4,where_helped___5,other_where_helped,social_contributions_complete
1,2,NaN,4/16/20 14:47,244.0,NaN,NaN,NaN,NaN,2.0,NaN,...,0,0,NaN,0,0,0,0,0,NaN,0
2,3,NaN,4/16/20 14:51,244.0,NaN,NaN,NaN,NaN,3.0,NaN,...,0,0,NaN,0,0,0,0,0,NaN,0
3,4,NaN,4/16/20 14:53,9.0,NaN,NaN,NaN,NaN,3.0,NaN,...,0,0,NaN,0,0,0,0,0,NaN,0
4,5,NaN,4/16/20 15:01,2.0,NaN,NaN,NaN,NaN,5.0,NaN,...,0,0,NaN,0,0,0,1,0,NaN,2
5,6,NaN,4/16/20 15:07,2.0,NaN,NaN,NaN,NaN,3.0,NaN,...,0,0,NaN,0,0,0,1,0,NaN,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3418,3419,NaN,3/7/21 21:37,2.0,32.0,NaN,NaN,NaN,7.0,1.0,...,0,0,NaN,0,0,0,0,0,NaN,2
3419,3420,NaN,4/5/21 12:20,1.0,NaN,5.0,NaN,NaN,3.0,4.0,...,0,0,NaN,0,0,0,0,0,NaN,2
3420,3421,NaN,4/5/21 12:59,2.0,21.0,NaN,NaN,NaN,2.0,1.0,...,0,0,NaN,0,0,0,0,0,NaN,0
3421,3422,NaN,4/5/21 13:35,243.0,NaN,NaN,NaN,London,3.0,1.0,...,0,0,NaN,0,0,0,0,0,NaN,2


In [9]:
# See all columns
my_list = list(stanford_df)

print(my_list)

['study_id', 'redcap_survey_identifier', 'introduction_timestamp', 'country', 'state_in_india', 'state_in_us', 'province_in_china', 'state_other_countries', 'age', 'race_and_ethnicity', 'asian_category', 'sex', 'gender_other', 'marital_status', 'living_with___1', 'living_with___2', 'living_with___3', 'living_with___4', 'living_with___5', 'living_with___6', 'living_with___7', 'living_with___8', 'living_with___9', 'total_household', 'education', 'income', 'work_status___1', 'work_status___2', 'work_status___3', 'work_status___4', 'work_status___6', 'work_status___7', 'work_status___9', 'work_status___10', 'unemployed_categories___1', 'unemployed_categories___2', 'unemployed_categories___3', 'employment_category', 'employmentcategory_other', 'essential', 'essential_worker_yes', 'nonessentialcategories', 'positive_covid', 'was_the_test', 'sample_for_test', 'quarantinelocation', 'household_positive___1', 'household_positive___2', 'household_positive___3', 'household_positive___4', 'househol

In [15]:
# Drop empty rows on some key fields we need to look at.
stanford_df.dropna(subset=["socio_emotional_overall", "mental_overall", "physical_overall", "physical_activities", "covid_exercise", "social_interactions_timestamp", "hobbies_and_health_timestamp", "social_contributions_timestamp"])

,study_id,redcap_survey_identifier,introduction_timestamp,country,state_in_india,state_in_us,province_in_china,state_other_countries,age,race_and_ethnicity,...,sector_of_contribution___4,sector_of_contribution___5,other,where_helped___1,where_helped___2,where_helped___3,where_helped___4,where_helped___5,other_where_helped,social_contributions_complete
39,40,NaN,4/20/20 11:19,1.0,NaN,5.0,NaN,NaN,6.0,1.0,...,1,0,NaN,1,0,1,0,0,NaN,2
40,41,NaN,4/20/20 11:32,1.0,NaN,5.0,NaN,NaN,2.0,1.0,...,0,0,NaN,0,0,1,0,0,NaN,2
41,42,NaN,4/20/20 11:40,1.0,NaN,1.0,NaN,NaN,2.0,1.0,...,0,0,NaN,0,1,0,0,0,NaN,2
43,44,NaN,4/20/20 18:03,1.0,NaN,5.0,NaN,NaN,3.0,2.0,...,0,1,I donated my time/efforts,1,0,0,1,1,In health care sector,2
46,47,NaN,4/20/20 19:49,1.0,NaN,5.0,NaN,NaN,7.0,8.0,...,1,1,srhrdjd,1,1,1,1,1,rshrdjdcv,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3415,3416,NaN,11/19/20 11:10,1.0,NaN,5.0,NaN,NaN,4.0,4.0,...,0,0,NaN,0,0,0,0,0,NaN,2
3416,3417,NaN,11/25/20 8:43,1.0,NaN,5.0,NaN,NaN,4.0,1.0,...,0,1,ordered food from local restarants,0,0,1,0,0,NaN,2
3418,3419,NaN,3/7/21 21:37,2.0,32.0,NaN,NaN,NaN,7.0,1.0,...,0,0,NaN,0,0,0,0,0,NaN,2
3419,3420,NaN,4/5/21 12:20,1.0,NaN,5.0,NaN,NaN,3.0,4.0,...,0,0,NaN,0,0,0,0,0,NaN,2
